In [34]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np
import ast
import requests

class Agent():
    def __init__(self) -> None:
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        self.model = AutoModel.from_pretrained('bert-base-uncased')

    def words2vector(self, sentence: str):
        sentence = "This is an example sentence."
        tokens = self.tokenizer.tokenize(sentence)

        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        input_ids = [self.tokenizer.cls_token_id] + input_ids + [self.tokenizer.sep_token_id]

        max_length = 64
        input_ids = input_ids + [self.tokenizer.pad_token_id] * (max_length - len(input_ids))
        input_tensor = torch.tensor([input_ids])

        with torch.no_grad():
            outputs = self.model(input_tensor)
            embedding = outputs[0][:, 0, :].numpy()

        return embedding[0]
    
model = Agent()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
qv = str(model.words2vector('Active ingredient').tolist())
qv

'[-0.6399794816970825, 0.21811088919639587, 0.3077543377876282, 0.6942612528800964, -0.209189310669899, -0.5834497213363647, 0.6039624214172363, -0.12885743379592896, 0.3099514842033386, -0.11490064114332199, -0.3697929382324219, -0.4513343870639801, -0.3282638192176819, 0.6700892448425293, 0.5967206358909607, 0.6667458415031433, -0.1763831377029419, 0.5179620385169983, 0.08317531645298004, -0.29886844754219055, 0.4232212007045746, 0.4656631350517273, 0.9299180507659912, 0.11039788275957108, -0.051615968346595764, -0.06626422703266144, -0.046358346939086914, -1.070488691329956, -0.5057221055030823, -0.025580422952771187, -0.4826614260673523, 0.534902811050415, 0.06846482306718826, -0.07219915091991425, 0.41488730907440186, -0.47224125266075134, -0.6704074740409851, -0.21100924909114838, 0.410153865814209, 0.20679286122322083, 0.02910652570426464, -0.6472131609916687, 0.832511842250824, 0.07199420034885406, -0.24872969090938568, -0.5784329771995544, -3.6920437812805176, 0.28061711788177

In [36]:
URL = 'http://localhost:8983/solr/ir_updated/select?indent=true&q.op=OR&&q={!knn f=feature_vector topK=10}' + qv
r = requests.get(url = URL)